# The Hubble Constant
25.05.27 AstroLab


작성자: Seungwu Yoo

## README

This notebook consists of three types of code cells:

The [Advanced] sections are dedicated to those who wish to further pursue and deeply study astronomy.

- [Mission]: Sections where you directly modify and manipulate values for practice.


- [Auto]: Sections that are not essential to manipulate for practice but are easy to understand.


- [Advanced]: Background information provided for practice; sections that you can execute without needing to fully understand or manipulate.

The [Advanced] sections are dedicated to those who wish to further pursue and deeply study astronomy.

### [Auto] Import the package

In [9]:
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

from astroquery.sdss import SDSS
from astropy import coordinates as coords

### [Advanced] Cosmicflows-4 Data

- Reference: https://ui.adsabs.harvard.edu/abs/2023ApJ...944...94T/abstract

- Data Source: Cosmicflows-4 Database

- The file 'allCF4_individual.txt' provides measurements for individual galaxies, including recessional velocities and distance moduli.

- Vcmb: The systemic velocity of the galaxy in the cosmic microwave background (CMB) reference frame.
    - This value accounts for the motion of the observer (e.g., the Earth, the Milky Way, or space-based telescopes like GAIA).
    - The dipole anisotropy observed in the CMB—caused by the observer's peculiar motion—can be used to correct observed redshifts, providing an isotropic frame of reference.
- DM (Distance Modulus): $m - M = 5\log_{10}\left(\frac{d}{pc}\right) - 5$

In [3]:
vcmb_CF4, dm_CF4, ra_CF4, dec_CF4 = np.loadtxt('allCF4_individual.txt', skiprows=5, unpack=True, usecols=[3, 4, -9,-8])

### [Advanced] Match the cosmicflows-4 data with SDSS query

- 'matching.txt' 파일을 만든 코드

- 시간이 오래 걸리므로 제공된 파일을 사용하는 것을 추천

- 사용하고 싶으면 맨 처음 '''와 맨 마지막 '''을 제거하고 실행

In [ ]:
'''from tqdm.notebook import tqdm

f = open('matching.txt', 'w')
f.write('ra_cf4 dec_cf4 ra_sdss dec_sdss r objid\n')
for ra, dec in tqdm(zip(ra_CF4, dec_CF4), total=len(ra_CF4), desc='Matching CF4 with SDSS'):
    # Define the coordinate object
    coord = coords.SkyCoord(ra, dec, unit="deg", frame='icrs')
    
    try:
        # Query for spectra within 2 arcseconds of the coordinate
        xid = SDSS.query_region(coord, spectro=True, radius='1arcsec', photoobj_fields=['r', 'objid'],
                            specobj_fields=['ra', 'dec', 'plate', 'mjd', 'fiberID'], timeout=300)
    
        f.write('%f %f %f %f %f %d %d %d %d\n'%(ra, dec, float(xid['ra'][0]), float(xid['dec'][0]),
                                     float(xid['r'][0]), int(xid['objid'][0]), int(xid['plate'][0]),
                                     int(xid['mjd'][0]), int(xid['fiberID'][0])))
    except:
        f.write(f'{ra} {dec} -999 -9999 -9 -9 -9 -9 -9\n')
        
f.close()'''

### [Advanced] Load 'matching.txt' file

- 'spectroscopy.ipynb'의 SDSS get_spectra() 함수에 넣을 plate, mjd, fiberid를 제공함

- Cosmicflows-4의 거리 (DM) 정보와 matching 시킬 수 있음

In [8]:
matched_ra, matched_dec, r, plate, mjd, fiberid = np.loadtxt('matching.txt', skiprows=1, 
                                                             unpack=True, usecols=[2,3,4,6,7,8])

print(f'plate: {plate[plate != -9]}')
print(f'mjd: {mjd[mjd != -9]}')
print(f'fiberid: {fiberid[fiberid != -9]}')

plate: [ 750. 6879.  751. ... 4534. 1929. 6305.]
mjd: [52235. 56539. 52251. ... 55863. 53349. 56563.]
fiberid: [605. 726. 329. ... 233. 597. 302.]


### [Advanced] Match the Cosmicflows-4 Data with SDSS Query

- The file 'matching.txt' was generated using the code below.

- Since this matching process can be time-consuming, $\color{magenta}\textbf{it is recommended to use the provided file directly.}$

- If you wish to re-run the code, simply remove the opening ''' and closing ''' to execute the block.

In [10]:
dist = pd.read_csv('dist.txt', sep=" ")
dist.head(5)

,spec,dist
0,0,57.623549
1,1,14.144904
2,2,59.951493
3,3,64.061927
4,4,28.933435


### [Mission] Calculate the Hubble constant

- Using the velocity–distance relation for at least 20 given spectra, estimate the Hubble constant.

- $\color{magenta}\textbf{You must generate and present the graph yourself.}$

- For linear regression, you may either use the formula introduced in lecture or apply a suitable statistical package.

- In the case where the intercept is zero ($Y = mX$):
    - $m=\frac{\sum_{i=1}^nY_iX_i}{\sum_{i=1}^nX_i^2}$

In [11]:
def linear_regression(xdat, ydat, intercept=True):
    XY, XX = 0, 0
    xbar, ybar = np.mean(xdat), np.mean(ydat)
    if intercept:
        ydat = ydat - ybar
        xdat = xdat - xbar
    for x, y in zip(xdat, ydat):
        XY += x * y
        XX += x * x
    m = XY / XX
    b = ybar - m * xbar if intercept else 0
    return m, b

### Additional thinking (Duty)

- (Discussion required!) Why might the Hubble constant you calculated differ from the known value?